In [ ]:
from google.colab import drive
drive.mount('/content/drive')
% cd /content/drive/MyDrive/HASOC Project Folder/2021 Dataset/CodeMix Dataset/

Mounted at /content/drive
/content/drive/MyDrive/HASOC Project Folder/2021 Dataset/CodeMix Dataset


In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

In [3]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt

In [4]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None) 

In [ ]:
from glob import glob
import re
import json

train_directories = []
for i in glob("data/train/*/"):
    for j in glob(i+'*/'):
        train_directories.append(j)

In [ ]:
data = []
for i in train_directories:
    with open(i+'data.json', encoding='utf-8') as f:
        data.append(json.load(f))
labels = []
for i in train_directories:
    with open(i+'labels.json', encoding='utf-8') as f:
        labels.append(json.load(f))

In [ ]:
def p2_flatten(d, l):

    flat_text = []

    flat_text.append({
        'tweet_id': d['tweet_id'],
        "context": d['tweet'],
        'text': d['tweet'],
        'label': l[d['tweet_id']]
    })

    for i in d['comments']:
        previous_tweet = ""

        flat_text.append({
            'tweet_id': i['tweet_id'],
            'context': d["tweet"],
            'text': i["tweet"],
            'label': l[i['tweet_id']]
        })

        if 'replies' in i.keys():
            for j in i['replies']:
                flat_text.append({
                    'tweet_id': j['tweet_id'],
                    'context': d["tweet"] + i["tweet"] + previous_tweet,
                    'text': j['tweet'],
                    'label': l[j['tweet_id']]
                })
                previous_tweet = previous_tweet + " [SEP] "+ j["tweet"]

    return flat_text

In [ ]:
data_label = []

for i in range(len(labels)):
    for j in p2_flatten(data[i], labels[i]):
        data_label.append(j)

In [ ]:
df = pd.DataFrame(data_label, columns = data_label[0].keys(), index = None)

In [ ]:
df.to_pickle("p2_flattened.pkl")

In [ ]:
df = df.head(25)

In [5]:
train_df=pd.read_pickle('p2_codemix_flat.pkl')
test_df=pd.read_pickle('p2_codemix_flat_test.pkl')

In [6]:
train_df["list_context"] = train_df["context"].map(lambda a: a.split(" [SEP] "))
test_df["list_context"] = test_df["context"].map(lambda a: a.split(" [SEP] "))

In [7]:
test_df = test_df.drop(['context'], axis = 1)
test_df = test_df.drop(['tweet_id'], axis = 1)


In [ ]:
test_df

In [8]:
train_df = train_df.drop(['context'], axis = 1)
train_df = train_df.drop(['tweet_id'], axis = 1)


In [9]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/cs18resch11003/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cs18resch11003/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def similarity_score(X,Y, distance=False):
  # tokenization
  X_list = word_tokenize(X) 
  Y_list = word_tokenize(Y)
    
  # sw contains the list of stopwords
  sw = stopwords.words('english') 
  l1 =[];l2 =[]
    
  # remove stop words from the string
  X_set = {w for w in X_list if not w in sw} 
  Y_set = {w for w in Y_list if not w in sw}
    
  # form a set containing keywords of both strings 
  rvector = X_set.union(Y_set) 
  for w in rvector:
      if w in X_set: l1.append(1) # create a vector
      else: l1.append(0)
      if w in Y_set: l2.append(1)
      else: l2.append(0)
  c = 0
  # cosine formula 
  for i in range(len(rvector)):
          c+= l1[i]*l2[i]
  cosine = c / float((sum(l1)*sum(l2))**0.5)
  
  if distance:
    return c
  return cosine

In [11]:
sims_df = []
for index in range(len(train_df)):
  sims = []
  normal_sims = []
  sum_sims = 0

  text = train_df.iloc[index]["text"]

  for context in train_df.iloc[index]["list_context"]:

    similarity = similarity_score(text, context, distance=True)

    if similarity==0:
      similarity = 0.001

    sum_sims = sum_sims+similarity
    
    sims.append(similarity)

  for i, sim in enumerate(sims):
    normal_sims.append(sims[i]/sum_sims)

  sims_df.append(sims)

In [12]:
train_df["weights"] = sims_df

In [ ]:
train_df

In [13]:
sims_df = []
for index in range(len(test_df)):
  sims = []
  normal_sims = []
  sum_sims = 0

  text = test_df.iloc[index]["text"]

  for context in test_df.iloc[index]["list_context"]:

    similarity = similarity_score(text, context, distance=True)

    if similarity==0:
      similarity = 0.001

    sum_sims = sum_sims+similarity
    
    sims.append(similarity)

  for i, sim in enumerate(sims):
    normal_sims.append(sims[i]/sum_sims)

  sims_df.append(sims)

In [14]:
test_df["weights"] = sims_df

In [ ]:
test_df

In [ ]:
!pip install transformers --quiet
!pip install bert-for-tf2 --quiet
!pip install tensorflow-text --quiet

In [1]:
import numpy as np
import torch
import pandas as pd
import transformers
from transformers import pipeline
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text
from bert import bert_tokenization
from scipy.spatial import distance
from sklearn.metrics import classification_report
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from transformers import DistilBertTokenizer, DistilBertModel, BertTokenizer, BertModel, AutoTokenizer, AutoModelForMaskedLM

In [16]:
class model:
    def __init__(self, df, model_name, avg_pooling=False):
        self.tokenizer = None
        self.model = None
        self.tokenized_padded_text = None
        self.attention_mask = None
        self.textip = None
        self.pooledOp = None
        self.input_dfs = None
        self.data_frame = df
        self.feature_df = None
        self.model_name = None
        self.InitModel(model_name, avg_pooling)

    def InitModel(self, model_name, avg_pooling):
      

        if model_name == 'distilBert':
            model_class, tokenizer_class, pretrained_weights = (
                DistilBertModel, DistilBertTokenizer, 'distilbert-base-uncased')
            self.tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
            self.model = model_class.from_pretrained(pretrained_weights)
            self.model_name = 'distilBert'

        if model_name == 'mBert':
            self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
            self.model = BertModel.from_pretrained("bert-base-multilingual-cased")
            self.max_seq_length = 512
            self.model_name = 'mBert'
        
        if model_name == 'mBert_p':
            #self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
            #self.model = BertModel.from_pretrained("bert-base-multilingual-cased")
            #self.max_seq_length = 512
            self.model_name = 'mBert_p'
            self.nlp = pipeline(task ="feature-extraction", model = 'bert-base-multilingual-cased', tokenizer='bert-base-multilingual-cased', framework='pt', device=0)


        if model_name == 'muril':
            self.textip = tf.keras.layers.Input(shape=(), dtype=tf.string)
            self.max_seq_length = 128
            muril_model, muril_layer = self.init_muril(
                model_url="https://tfhub.dev/google/MuRIL/1", max_seq_length=self.max_seq_length,
                avg_pooling=avg_pooling)
            vocab_file = muril_layer.resolved_object.vocab_file.asset_path.numpy()
            do_lower_case = muril_layer.resolved_object.do_lower_case.numpy()
            self.tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)
            self.model_name = 'muril'
            self.model = muril_model
            self.avg_pooling = avg_pooling
        if model_name=='xlmr':    
            self.model_name = 'xlmr'
            self.nlp = pipeline(task ="feature-extraction", model = 'xlm-roberta-base', tokenizer='xlm-roberta-base', framework='pt', device=0)
            self.avg_pooling = avg_pooling 
            
    def tokenize(self, column):
        tokenized_text = column.apply((lambda x: self.tokenizer.encode(x,truncation=True,add_special_tokens=True)))
        max_len = 0
        for i in tokenized_text.values:
            if len(i) > max_len:
                max_len = len(i)
        self.tokenized_padded_text = np.array([i + [0]*(max_len-len(i)) for i in tokenized_text.values])
        self.create_attention_mask()

    def create_attention_mask(self):
        self.attention_mask = np.where(self.tokenized_padded_text != 0, 1, 0)
        print(type(self.tokenized_padded_text))
        self.input_ids = torch.tensor(self.tokenized_padded_text)
        self.attention_mask = torch.tensor(self.attention_mask)

    def init_muril(self, model_url, max_seq_length, avg_pooling):
        inputs = dict(
            input_word_ids=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
            input_mask=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
            input_type_ids=tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32),
        )

        muril_layer = hub.KerasLayer(model_url, trainable=True)
        outputs = muril_layer(inputs)
        print(outputs)
        assert 'sequence_output' in outputs
        assert 'pooled_output' in outputs
        assert 'encoder_outputs' in outputs
        assert 'default' in outputs
        if avg_pooling:
            return tf.keras.Model(inputs=inputs, outputs=outputs["encoder_outputs"]), muril_layer
        else:
            return tf.keras.Model(inputs=inputs, outputs=outputs["pooled_output"]), muril_layer

    def create_input(self, input_strings, tokenizer, max_seq_length):
        input_ids_all, input_mask_all, input_type_ids_all = [], [], []
        for input_string in input_strings:
            input_tokens = ["[CLS]"] + tokenizer.tokenize(input_string) + ["[SEP]"]
            input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
            sequence_length = min(len(input_ids), max_seq_length)

            if len(input_ids) >= max_seq_length:
                input_ids = input_ids[:max_seq_length]
            else:
                input_ids = input_ids + [0] * (max_seq_length - len(input_ids))

            input_mask = [1] * sequence_length + [0] * (max_seq_length - sequence_length)

            input_ids_all.append(input_ids)
            input_mask_all.append(input_mask)
            input_type_ids_all.append([0] * max_seq_length)

        return np.array(input_ids_all), np.array(input_mask_all), np.array(input_type_ids_all)

    def encode(self, input_text):
        input_ids, input_mask, input_type_ids = self.create_input(input_text,
                                                                  self.tokenizer,
                                                                  self.max_seq_length)
        inputs = dict(
            input_word_ids=input_ids,
            input_mask=input_mask,
            input_type_ids=input_type_ids,
        )
        return self.model(inputs)

    def GetFeatures(self, input=None):
        if self.model_name == 'mBert':
            with torch.no_grad():
                last_hidden_states = self.model(self.input_ids, attention_mask=self.attention_mask)
            last_hidden_states['last_hidden_state'].size()
            self.features = last_hidden_states[0][:, 0, :].numpy()
            self.features = pd.DataFrame(self.features)
        elif self.model_name == 'muril':
            embeddings = self.encode(input)
            if not self.avg_pooling:
                self.features = pd.DataFrame(embeddings.numpy())
            else:
                f1 = embeddings[7][:, 0, :].numpy()
                f2 = embeddings[6][:, 0, :].numpy()
                f3 = embeddings[5][:, 0, :].numpy()
                self.features = pd.DataFrame((f1+f2+f3)/3)
        elif self.model_name == 'xlmr':
            sentences=input
            features = self.nlp(sentences, truncation=True) 
            featurelist=list()
            for i in features:
               featurelist.append(i[0][0])
            self.features=pd.DataFrame(featurelist)     
        elif self.model_name == 'mBert_p':
            sentences=input
            features = self.nlp(sentences, truncation=True) 
            featurelist=list()
            for i in features:
               featurelist.append(i[0][0])
            self.features=pd.DataFrame(featurelist)           
        return self.features

In [2]:
class classifiers:

  def __init__(self,features_train,label_train,features_test,label_test):
    #self.features_set=features
    #self.labels=label
    #self.Createstaticsplit(features,label)
    self.train_features=features_train
    self.train_labels=label_train
    self.test_features=features_test
    self.test_labels=label_test
    self.accuracy=list()
    self.f1score=list()
    self.models=list()
    self.y_pred=list()

  def classify(self,svm=True,random_forest=True,xgboost=True,logistic_regression=True,ann=True)  :
      if svm==True:
        acc,f1_score=self.CreateSVMClassifier()
        self.accuracy.append(acc)
        self.f1score.append(f1_score)
        
        self.models.append('svm')
      if random_forest==True:
        acc,f1_score,y=self.RandomForestClassifier()
        self.accuracy.append(acc)
        self.f1score.append(f1_score)
        self.y_pred.append(y)
        self.models.append('random_forest')
      if xgboost==True:
        acc,f1_score,y=self.XGBClassifier(2)
        self.accuracy.append(acc)
        self.f1score.append(f1_score)
        self.y_pred.append(y)
        self.models.append('xgboost')
      if logistic_regression==True:
        acc,f1_score,y=self.LogisticRegression()
        self.accuracy.append(acc)
        self.f1score.append(f1_score)
        self.y_pred.append(y)
        self.models.append('lr')
      if ann==True:
        acc,f1_score=self.annClassifier()
        self.accuracy.append(acc)
        self.f1score.append(f1_score)
        self.models.append('ann')
      return self.accuracy,self.f1score,self.models,self.y_pred

  def MajorityVotingClassifier(self, num_class):
        acc_xg, f1_xg, y_xgboost = self.XGBClassifier(num_class)
        acc_rf, f1_rf, y_rf = self.RandomForestClassifier()
        acc_rf, f1_rf, y_lr = self.LogisticRegression()
        y_pred = list()
        for i in range(len(y_xgboost)):
            preds = list()
            preds.append(y_xgboost[i])
            preds.append(y_rf[i])
            preds.append(y_lr[i])
            y_pred.append(max(set(preds), key=preds.count))
        cm=confusion_matrix(self.test_labels,y_pred)    
        return accuracy_score(self.test_labels, y_pred), f1_score(self.test_labels, y_pred, average='macro') ,cm,y_pred 

  def VotingClassifier(self):
      clf1 = LogisticRegression(multi_class='multinomial', random_state=1)
      clf2 = RandomForestClassifier(n_estimators=50, random_state=1)   
      clf3=XGBClassifier(n_estimators=500,learning_rate=1, max_depth=2,objective='multi:softmax',num_class=2) 
      eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('xgb', clf3)], voting='hard') 
      eclf.fit(self.train_features,self.train_labels)
      y_pred= eclf.predict(self.test_features) 
      return print(accuracy_score(self.test_labels, y_pred)),print(f1_score(self.test_labels, y_pred,average='macro')),y_pred

  def XGBClassifier(self,num_class):
     from xgboost import XGBClassifier
     classifier = XGBClassifier(n_estimators=500,learning_rate=1, max_depth=2,objective='multi:softmax',num_class=num_class)
     classifier.fit(self.train_features, self.train_labels)
     
     y_pred = classifier.predict(self.test_features)
     return print(accuracy_score(self.test_labels, y_pred)),print(f1_score(self.test_labels, y_pred,average='macro')),y_pred

  def CreateSVMClassifier(self):
    from sklearn.svm import SVC
    classifier = SVC(kernel = 'poly',decision_function_shape='ovr', random_state = 0)
    classifier.fit(self.train_features, self.train_labels)
  
    y_pred = classifier.predict(self.test_features)
    return accuracy_score(self.test_labels, y_pred),f1_score(self.test_labels, y_pred,average='macro')

  def RandomForestClassifier(self):
     from sklearn.ensemble import RandomForestClassifier
     classifier = RandomForestClassifier(n_estimators = 500, criterion = 'entropy', random_state = 0)
     classifier.fit(self.train_features, self.train_labels)

     y_pred = classifier.predict(self.test_features)
     return print(accuracy_score(self.test_labels, y_pred)),print(f1_score(self.test_labels, y_pred,average='macro')),y_pred

  def Createstaticsplit(self,features,labels,split_per=0.8):
   num=np.shape(features)[0]
   self.train_features=features.head(int(split_per*num))
   self.train_labels=labels.head(int(split_per*num))
   self.test_features=features.tail(num-int(split_per*num))
   self.test_labels=labels.tail(num-int(split_per*num))
  def annClassifier(self):
      import tensorflow as tf
      from sklearn.compose import ColumnTransformer
      from sklearn.preprocessing import OneHotEncoder
      ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
      feature_set = np.array(ct.fit_transform(self.features_set))
      #self.labels.replace('NOT',0,inplace=True)
      #self.labels.replace('HOF',1,inplace=True)
      train_features, test_features, train_labels, test_labels = train_test_split(feature_set, self.labels)

      ann = tf.keras.models.Sequential()
      ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
      ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
      ann.add(tf.keras.layers.Dense(units=1, activation='softmax'))
      ann.add(tf.keras.layers.Dense(units=1, activation='softmax'))
      ann.add(tf.keras.layers.Dense(units=1, activation='softmax'))
      ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
      ann.fit(train_features, train_labels, batch_size = 32, epochs = 200)

      y_pred = ann.predict(test_features)
     # y_pred = (y_pred > 0.5)
      return accuracy_score(test_labels, y_pred),f1_score(test_labels, y_pred,average='macro')

  def LogisticRegression(self):
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression 
    lr_clf = LogisticRegression(multi_class='multinomial')
    lr_clf.fit(self.train_features, self.train_labels) 

    y_pred=lr_clf.predict(self.test_features)
    return print(accuracy_score(self.test_labels, y_pred)),print(f1_score(self.test_labels, y_pred,average='macro')),y_pred

In [3]:
def classify(train_dataframe,test_dataframe,classifier):
   classifier_class=classifiers(features_train=train_dataframe.iloc[:, :-1],label_train=train_dataframe.label,features_test=test_dataframe.iloc[:, :-1],label_test=test_dataframe.label)
   test_labels=test_dataframe.label
   if classifier == 'A':
    accuracies,f1_scores,models,y=classifier_class.classify(svm=True,random_forest=True,xgboost=True,logistic_regression=True,ann=False)
    for i in range(len(y)) :
      print(models[i])
      print(classification_report(test_labels, y[i], labels=[0,1]))
    a,f,cm,y_pred=classifier_class.MajorityVotingClassifier(2)  
    print("VC")
    print(classification_report(test_labels, y_pred, labels=[0,1]))   
   elif classifier == 'LR':
      accuracies,f1_scores,models,y=classifier_class.classify(svm=False,random_forest=False,xgboost=False,logistic_regression=True,ann=False)
      print(classification_report(test_labels, y[0], labels=[0,1]))
   elif classifier == 'RF':  
      accuracies,f1_scores,models,y=classifier_class.classify(svm=False,random_forest=True,xgboost=False,logistic_regression=False,ann=False)
      print(classification_report(test_labels, y[0], labels=[0,1]))
   elif classifier == 'XGBOOST':
      accuracies,f1_scores,models,y=classifier_class.classify(svm=False,random_forest=False,xgboost=True,logistic_regression=False,ann=False)
      print(classification_report(test_labels, y[0], labels=[0,1]))
   elif classifier == 'M_VC': 
      a,f,cm,y_pred=classifier_class.MajorityVotingClassifier(2)  
      print(classification_report(test_labels, y_pred, labels=[0,1])) 
   elif classifier == 'VC': 
      a,f,y_pred=classifier_class.VotingClassifier()  
      print(classification_report(test_labels, y_pred, labels=[0,1])) 

In [19]:
model_pipeline=model(train_df,model_name='mBert_p')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
df.sample(2)

In [20]:
from operator import add

row_embeddings = []
for weight,context in zip(train_df["list_context"], train_df["weights"]):
  final_embeddings = np.zeros(768)
  for sub_weight, sub_context in zip(context, weight):
    text_embeddings = model_pipeline.GetFeatures(sub_context).iloc[0].to_list()
    text_embeddings = list(map(lambda x:sub_weight*x, text_embeddings))
    final_embeddings  = list( map(add, final_embeddings, text_embeddings) )

  row_embeddings.append(final_embeddings)

/raid/cs18resch11003/anaconda3/envs/aditi/lib/python3.8/site-packages/transformers/pipelines/base.py:1075: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [21]:
train_df["context_embeddings"] = row_embeddings

In [22]:
row_embeddings = []
for text in train_df["text"]:
  text_embeddings = model_pipeline.GetFeatures(text).iloc[0].to_list()
  row_embeddings.append(text_embeddings)

In [23]:
train_df["text_embeddings"] = row_embeddings

In [24]:
from operator import add

row_embeddings = []
for context,text in zip(train_df["context_embeddings"], train_df["text_embeddings"]):
  final_embeddings = np.zeros(768)

  final_embeddings  = list(map(add, context, text) )

  row_embeddings.append(final_embeddings)

train_df["training_embeddings"] = row_embeddings

In [ ]:
train_df.sample(5)

In [69]:
train_df["label"].unique()

array([1, 0])

In [25]:
TASK1 = {"HOF": 1, "NONE": 0}

In [26]:
train_df['label']=train_df['label'].map(TASK1)

In [27]:
train_X = pd.DataFrame(train_df['training_embeddings'].tolist())

In [28]:
train_y = train_df["label"]

In [29]:
model_pipeline=model(test_df,model_name='mBert_p')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
from operator import add

row_embeddings = []
for weight,context in zip(test_df["list_context"], test_df["weights"]):
  final_embeddings = np.zeros(768)
  for sub_weight, sub_context in zip(context, weight):
    text_embeddings = model_pipeline.GetFeatures(sub_context).iloc[0].to_list()
    text_embeddings = list(map(lambda x:sub_weight*x, text_embeddings))
    final_embeddings  = list( map(add, final_embeddings, text_embeddings) )

  row_embeddings.append(final_embeddings)
test_df["context_embeddings"] = row_embeddings  

In [31]:
row_embeddings = []
for text in test_df["text"]:
  text_embeddings = model_pipeline.GetFeatures(text).iloc[0].to_list()
  row_embeddings.append(text_embeddings)
test_df["text_embeddings"] = row_embeddings  

In [32]:
from operator import add
row_embeddings = []
for context,text in zip(test_df["context_embeddings"], test_df["text_embeddings"]):
  final_embeddings = np.zeros(768)
  final_embeddings  = list(map(add, context, text) )
  row_embeddings.append(final_embeddings)
test_df["training_embeddings"] = row_embeddings

In [33]:
test_df['label']=test_df['label'].map(TASK1)

In [34]:
test_X = pd.DataFrame(test_df['training_embeddings'].tolist())

In [35]:
test_y = test_df["label"]

In [36]:
train_X['label']=train_y
test_X['label']=test_y

In [37]:
train_X.to_pickle('mbert_p2_train.pkl')
test_X.to_pickle('mbert_p2_test.pkl')

In [ ]:
train_df['label']

In [38]:
classify(train_X,test_X,'RF')

0.5066344993968637
0.5043728667382931
              precision    recall  f1-score   support

           0       0.41      0.54      0.47       334
           1       0.61      0.48      0.54       495

    accuracy                           0.51       829
   macro avg       0.51      0.51      0.50       829
weighted avg       0.53      0.51      0.51       829



In [39]:
classify(train_X,test_X,'VC')

/raid/cs18resch11003/anaconda3/envs/aditi/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:12:52] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.4957780458383595
0.4948776704916503
0.5066344993968637
0.5043728667382931
0.4077201447527141
0.3009385760779027
              precision    recall  f1-score   support

           0       0.41      0.64      0.50       334
           1       0.61      0.38      0.46       495

    accuracy                           0.48       829
   macro avg       0.51      0.51      0.48       829
weighted avg       0.53      0.48      0.48       829



In [43]:
classify(train_X,test_X,'LR')

0.4077201447527141
0.3009385760779027
              precision    recall  f1-score   support

           0       0.40      0.99      0.57       334
           1       0.70      0.01      0.03       495

    accuracy                           0.41       829
   macro avg       0.55      0.50      0.30       829
weighted avg       0.58      0.41      0.25       829



In [41]:
classify(train_X,test_X,'XGBOOST')

/raid/cs18resch11003/anaconda3/envs/aditi/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:14:03] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.4957780458383595
0.4948776704916503
              precision    recall  f1-score   support

           0       0.41      0.56      0.47       334
           1       0.60      0.45      0.52       495

    accuracy                           0.50       829
   macro avg       0.51      0.51      0.49       829
weighted avg       0.53      0.50      0.50       829



In [32]:
train_df=pd.read_pickle('mbert_p1_train.pkl')
test_df=pd.read_pickle('mbert_p1_test.pkl')

In [33]:
classify(train_df,test_df,'VC')

/raid/cs18resch11003/anaconda3/envs/aditi/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/raid/cs18resch11003/anaconda3/envs/aditi/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecat

[22:37:59] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.5971049457177322
0.5778610545317056
              precision    recall  f1-score   support

           0       0.66      0.68      0.67       495
           1       0.50      0.48      0.49       334

    accuracy                           0.60       829
   macro avg       0.58      0.58      0.58       829
weighted avg       0.59      0.60      0.60       829



In [34]:
train_df=pd.read_pickle('muril_p1_train.pkl')
test_df=pd.read_pickle('muril_p1_test.pkl')

In [35]:
classify(train_df,test_df,'VC')

/raid/cs18resch11003/anaconda3/envs/aditi/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:39:10] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.51145958986731
0.5111152853192614
              precision    recall  f1-score   support

           0       0.63      0.45      0.52       495
           1       0.42      0.60      0.50       334

    accuracy                           0.51       829
   macro avg       0.53      0.53      0.51       829
weighted avg       0.55      0.51      0.51       829



In [4]:
train_df=pd.read_pickle('muril_p2_train.pkl')
test_df=pd.read_pickle('muril_p2_test.pkl')

In [5]:
classify(train_df,test_df,'VC')

/raid/cs18resch11003/anaconda3/envs/aditi/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:38:27] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [ ]:
classify(train_df,test_df,'RF')

In [ ]:
classify(train_df,test_df,'M_VC')

In [ ]:
classify(train_df,test_df,'LR')

In [ ]:
classify(train_df,test_df,'XGBOOST')

In [ ]:
params_xgboost = {"n_estimators": 500, "learning_rate": 1, "max_depth": 2,
                  "objective": 'multi:softmax', "num_class": 4}
params_lr = {"multi_class": 'multinomial'}
params_rf = {"n_estimators": 500, "criterion": 'entropy', "random_state": 0}
TRAIN_RATIO = 0.8

In [ ]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

class Classifiers:
    def __init__(self, features, label):
        self.features_set = features
        self.labels = label
        self.Createstaticsplit(features, label)
        self.accuracy = list()
        self.f1score = list()

    def MajorityVotingClassifier(self, num_class):
        acc_xg, f1_xg, y_xgboost = self.XGBClassifier(num_class)
        acc_rf, f1_rf, y_rf = self.RandomForestClassifier()
        acc_rf, f1_rf, y_lr = self.LogisticRegression()
        y_pred = list()
        for i in range(len(y_xgboost)):
            preds = list()
            preds.append(y_xgboost[i])
            preds.append(y_rf[i])
            preds.append(y_lr[i])
            y_pred.append(max(set(preds), key=preds.count))
        return accuracy_score(
            self.test_labels, y_pred), f1_score(
            self.test_labels, y_pred, average='weighted')

    def XGBClassifier(self, num_class):
        classifier = XGBClassifier(**params_xgboost)
        classifier.fit(self.train_features, self.train_labels)
        y_pred = classifier.predict(self.test_features)
        return accuracy_score(self.test_labels, y_pred), f1_score(self.test_labels, y_pred, average='weighted'), y_pred

    def RandomForestClassifier(self):
        classifier = RandomForestClassifier(**params_rf)
        classifier.fit(self.train_features, self.train_labels)
        y_pred = classifier.predict(self.test_features)
        return accuracy_score(self.test_labels, y_pred), f1_score(self.test_labels, y_pred, average='weighted'), y_pred

    def Createstaticsplit(self, features, labels, split_per=0.8):
        num = np.shape(features)[0]
        self.train_features = features.head(int(split_per*num))
        self.train_labels = labels.head(int(split_per*num))
        self.test_features = features.tail(num-int(split_per*num))
        self.test_labels = labels.tail(num-int(split_per*num))

    def LogisticRegression(self):
        lr_clf = LogisticRegression(**params_lr)
        lr_clf.fit(self.train_features, self.train_labels)
        y_pred = lr_clf.predict(self.test_features)
        return accuracy_score(self.test_labels, y_pred), f1_score(self.test_labels, y_pred, average='weighted'), y_pred

In [ ]:
handler=Classifiers(features=X,label=y)

accuracies,f1_scores=handler.MajorityVotingClassifier(2)

print("Accuracy: {}, F1 Score: {}".format(accuracies, f1_scores))